In [4]:
import chess
import pandas as pd
import stockfish
import math

In [7]:
kaggle_fens_dir = "/Users/diegoesparza/CS_Ventures/current_projects/chess_engine_using_probabilities/kaggle_fens"
sf = stockfish.Stockfish(path="/Users/diegoesparza/Downloads/stockfish/stockfish-macos-x86-64")
sf.resume_full_strength()

In [8]:
openings = pd.read_csv(kaggle_fens_dir + "/openings.csv")

In [25]:
too_short = 0
not_equal = 1
equal_low = 2
equal_mid = 3
equal = 4
def of_eval(eval):
    if eval["type"] == "cp":
        score = eval["value"]
        if abs(score) < 150:
            return equal
        else:
            return not_equal
    else:
        return not_equal
    
def eval_board(b, eval_time):
    fen = b.fen()
    sf.set_fen_position(fen, send_ucinewgame_token=True)
    eval = sf.get_evaluation(searchtime=eval_time)
    return of_eval(eval)

def fen_after_opening(moves, sf):
    min_length = 16
    mid_length = 20
    max_length = 24
    if len(moves) < min_length:
        return None
    
    pushed = 0
    b = chess.Board()
    for move in moves[:max_length]:
        pushed += 1
        b.push_san(move)
        
    res = eval_board(b, 100)
    if res == equal:
        return b.fen()
    
    if pushed >= mid_length:
        for _ in range(pushed - mid_length):
            pushed -= 1
            b.pop()
    
        if eval_board(b, 50) == equal:
            return b.fen()
    
    # assume pushed is bigger than min_length, otherwise we'd have returned
    # None
    for _ in range(pushed - min_length):
        pushed -= 1
        b.pop()
    
    if eval_board(b, 30) == equal:
        return b.fen()
    return None

In [27]:
fens = []
for index, row in openings.iterrows():
    moves = row["moves"].split(" ")
    fen = fen_after_opening(moves, sf)
    if fen is not None:
        fens.append(fen)
        
    if index % 50 == 0:
        print(f"processed {index}, found {len(fens)}")

processed 0, found 0
processed 50, found 27
processed 100, found 65
processed 150, found 103
processed 200, found 146
processed 250, found 185
processed 300, found 218
processed 350, found 259
processed 400, found 290
processed 450, found 329
processed 500, found 369
processed 550, found 401
processed 600, found 435
processed 650, found 473
processed 700, found 511
processed 750, found 548
processed 800, found 592
processed 850, found 635
processed 900, found 668
processed 950, found 695
processed 1000, found 731
processed 1050, found 771
processed 1100, found 811
processed 1150, found 855
processed 1200, found 894
processed 1250, found 920
processed 1300, found 950
processed 1350, found 977
processed 1400, found 1014
processed 1450, found 1048
processed 1500, found 1084
processed 1550, found 1120
processed 1600, found 1146
processed 1650, found 1176
processed 1700, found 1201
processed 1750, found 1242
processed 1800, found 1268
processed 1850, found 1291
processed 1900, found 1331
pr

In [28]:
fens = list(set(fens))
print(len(fens))

13124


In [29]:
with open(kaggle_fens_dir + "/strong_openings", "w+") as f:
    for fen in fens:
        f.write(f"{fen}\n")